In [25]:
from elasticsearch import Elasticsearch
es = Elasticsearch()
es.xpack.security.enable = True

mapping = {
    "mappings":{
        "properties":{
        "Subject":{
            "type":"text"
        },
        "From":{
            "type":"text"
        },
        "To":{
            "type":"text"
        },
        "Date":{
            "type":"date",
            "format":"yyyy-MM-dd HH:mm:ss"
        },
        "Messsage_ID":{
            "type":"text"
        },
        "Body":{
            "type":"text"
        }
        
        }
        }
}

In [32]:
es.indices.create(index = "final_index", mappings= mapping["mappings"])

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'final_index'}

In [158]:
import email, imaplib
from email.header import decode_header
import credentials
from datetime import datetime, timedelta
from email.utils import parsedate_tz, mktime_tz
x = 0
try:
    search_params ={
        "_source":{
        "includes":["id", "Subject", "From", "To", "Date","Body"]
        },"query":{
            "match":{
                '_id':"final element uid"
            }
        }
    }
    x = es.search(index = "final_index", body = search_params)
    x = x["hits"]["hits"][0]["_source"]["Subject"]
except: pass

sel = imaplib.IMAP4_SSL("imap.gmail.com")
sel.login(credentials.mail_id(), credentials.password())
sel.select('INBOX')
_, msg = sel.uid('search', None ,'ALL' )    
latest_email_uid = msg[0].split()
latest_email_uid.reverse()
print(latest_email_uid[0])
for num in latest_email_uid: 
    _id = num.decode('utf-8')
    if int(_id) > x1:
        response, data = sel.uid('fetch', num, '(RFC822)')
        raw_email = data[0][1]
        d = {}
        msg = email.message_from_bytes(raw_email)
        subject, encoding = decode_header(msg["Subject"])[0]
        try:
            subject = subject.decode(encoding)
        except:pass
        if isinstance(subject,bytes):
             subject = subject.decode('utf-8')
        try:
            to = decode_header(msg.get("To"))[0][0]
            if isinstance(to , bytes):
                to = to.decode('utf-8')
        except: pass
        From , encoding = decode_header(msg.get("From"))[0]
        if isinstance(From ,bytes):
            try:
                From = From.decode(encoding)
            except:
                From = From.decode('utf-8')
        try:
            date = decode_header(msg["Date"])[0][0]
            timestamp = mktime_tz(parsedate_tz(date))
            utc_time = datetime(1970, 1, 1) + timedelta(seconds=timestamp)
            date = str(utc_time)
        except: pass
        d["Subject"]= subject
        d["To"]=to
        d["From"]=From
        d["Date"]= date
        if msg.is_multipart():
            for part in msg.walk():
                content_type = part.get_content_type()
                content_disposition = str(part.get("Content-Disposition"))
                try:
                    body = part.get_payload(decode=True).decode()
                except: pass
                if content_type == "text/plain" and "attachment" not in content_disposition:
                    d["Body"] = body
        else:
            content_type = msg.get_content_type()
            try:
                body = msg.get_payload(decode=True).decode()
            except:  pass
            if content_type=="text/plain":
                d["Body"] = body
        es.index(index = "final_index" ,id = _id, document = d)
    else:
        break
last_uid = latest_email_uid[0]
last_uid = last_uid.decode("utf-8")
es.index(index = "final_index", id = "final element uid", body = {"Subject":last_uid})

sel.logout()


C:\Users\HP\AppData\Local\Temp/ipykernel_25936/3825651773.py:17: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  x = es.search(index = "final_index", body = search_params)


b'2458'


C:\Users\HP\AppData\Local\Temp/ipykernel_25936/3825651773.py:83: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index = "final_index", id = "final element uid", body = {"Subject":last_uid})


('BYE', [b'LOGOUT Requested'])

In [163]:
s = es.search(index = "final_index")
s

{'took': 12,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2058, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'final_index',
    '_type': '_doc',
    '_id': '1',
    '_score': 1.0,
    '_source': {'Subject': 'Finish setting up your new Google account',
     'To': 'keshavvenkat22@gmail.com',
     'From': 'Google Community Team <googlecommunityteam-noreply@google.com>',
     'Date': '2020-02-21 13:39:16',
     'Body': "\r\nGoogle  \r\n<https://www.google.com/appserve/mkt/p/AFnwnKUUShJuDYHooUekjAA4EqVLmckv1ImWDLAVY4o988f4lq83MlPkA_L3kdKqzfs_3juWOoKNo0RD822Pb_yg82Q8blE-4EfGF9aPloshmGeGRXLq1XmyalLem0dhjKZELMhnex3VclfDDaAkd-5k46j8oBg_PV6P5FA>\r\n\r\nbanner\r\n\r\nHi Keshav,\r\n\r\nWelcome to Google. Your new account comes with access to Google products,  \r\napps and services.\r\n\r\nHere's a few tips to get you started.\r\n\r\nGoogle Go\r\n\r\nGet a lighter, faster way to search\r\n\r\nSearch in a fast

In [162]:
search_params ={
    "_source":{
    "includes":["id", "Subject", "From", "To", "Date","Body"]
    },"query":{
        "match":{
            '_id':"final element uid"
        }
    }
}
x = es.search(index = "final_index", body = search_params)
x

C:\Users\HP\AppData\Local\Temp/ipykernel_25936/3725552757.py:10: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  x = es.search(index = "final_index", body = search_params)


{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'final_index',
    '_type': '_doc',
    '_id': 'final element uid',
    '_score': 1.0,
    '_source': {'Subject': '2458'}}]}}